In [1]:
import db_functions
import pandas as pd
import numpy as np
import re

from sqlalchemy import MetaData, Table, Column, ForeignKey, Integer
from sqlalchemy import Boolean
from sqlalchemy.orm import mapper

In [ ]:
sql_engine = db_functions.connect_to_database_sqlalchemy()
df_amenities = pd.read_sql(sql='SELECT id, amenities FROM `dados-imoveis`', con=sql_engine).set_index('id')
df_amenities
list_amenities = []
df_amenities = df_amenities['amenities'].str.lower().str.replace(' ', '_').str.split(pat='\n', expand=True)

for i_lin in df_amenities.values:
	for i_amenity in i_lin:
		if i_amenity not in list_amenities and type(i_amenity) == str:
			list_amenities.append(i_amenity)
			
list_amenities.sort()
list_amenities
metadata_houses = MetaData()
column_types = [Boolean for i in range(len(list_amenities))]
houses_table  = Table('dados-imoveis', metadata_houses, (
	Column('id', Integer, primary_key = True)
))

# Adding columns

columns = [
	Column('id', Integer, primary_key = True, unique=True),
	Column('house_id', Integer, ForeignKey(houses_table.c.id), nullable=False, unique=True)
]

for index, (name, type) in enumerate(zip(list_amenities, column_types)):
			columns.append(Column(name, type))

metadata_amenities = MetaData()
amenities_table  = Table('amenities', metadata_amenities, *columns)
metadata_amenities.create_all(sql_engine)
dict_amenities = dict(zip(list_amenities, range(0,len(list_amenities))))

list_to_update = np.array([])
df_amenities = df_amenities.replace(dict_amenities)
df_amenities = df_amenities.astype('Int64')

for index, row in df_amenities.replace(dict_amenities).iterrows():
	temp_list = np.full(len(list_amenities), 'False')
	for i_amenity in row:
		if not pd.isna(i_amenity):
			temp_list[i_amenity] = 'True'
	temp_list = np.insert(temp_list, 0, index)
	if len(list_to_update) > 0:
		list_to_update = np.vstack((list_to_update, temp_list.copy()))
	else:
		list_to_update = temp_list.copy()
insert_query = f'INSERT INTO `amenities` (`house_id`, `{"`, `".join(list_amenities)}`)\n'
insert_query += f'VALUES\n'

for lin in list_to_update:
    subbed = re.sub(string=str(lin), pattern='(\' \'|\'\n \'|\' \n\')', repl=', ')
    insert_query += f'{chr(9)}(' + re.sub(pattern='(?:\n|\[|\]|\')', repl='', string=subbed) + '),\n'
    
insert_query =  re.sub(pattern=',(?!,)$', repl=';', string=insert_query)
db, db_cursor = db_functions.connect_to_database_mysql_connector()

db_cursor.execute(insert_query)
db.commit()